SIR MODEL

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random

def initialize_population_graph(num_nodes, distance):
    G = nx.random_geometric_graph(num_nodes, distance, seed=896803)
    return G

def initialize_node_states(G, initial_infected_fraction):
    num_infected = int(initial_infected_fraction * G.number_of_nodes())
    node_list = list(G.nodes())
    infected_nodes = random.sample(node_list, num_infected)
    node_states = {node: 'I' if node in infected_nodes else 'S' for node in G.nodes()}
    return node_states

def plot_sir_graph(G, node_states, title):
    plt.figure(figsize=(16, 16))
    pos = nx.get_node_attributes(G, "pos")

    colors = {'S': 'blue', 'I': 'red', 'R': 'green'}
    node_colors = [colors[node_states[node]] for node in G.nodes()]

    edge_colors = [colors[node_states[u]] for u, v in G.edges()]

    nx.draw_networkx_edges(G, pos, alpha=0.2, edge_color="black")
    nx.draw_networkx_nodes(
        G,
        pos,
        node_color=node_colors,
        node_size=15,
        alpha=0.8
    )
    nx.draw_networkx_edges(G, pos, alpha=0.2, edge_color=edge_colors)

    plt.xlim(-0.05, 1.05)
    plt.ylim(-0.05, 1.05)
    plt.title(title)
    plt.axis("off")
    plt.show()

def infect_neighbors(G, node_states, beta):
    new_node_states = node_states.copy()
    for node in G.nodes():
        if node_states[node] == 'I':
            for neighbor in G.neighbors(node):
                if node_states[neighbor] == 'S' and random.random() < beta:
                    new_node_states[neighbor] = 'I'
    return new_node_states

def recover_nodes(node_states, gamma):
    new_node_states = node_states.copy()
    for node in node_states:
        if node_states[node] == 'I' and random.random() < gamma:
            new_node_states[node] = 'R'
    return new_node_states

def SIR_model_visualization(G, beta, gamma, initial_infected_fraction, num_steps):
    node_states = initialize_node_states(G, initial_infected_fraction)

    for step in range(num_steps):
        plot_sir_graph(G, node_states, f'Step {step + 1}')
        node_states = infect_neighbors(G, node_states, beta)
        node_states = recover_nodes(node_states, gamma)

if __name__ == "__main__":
    num_nodes = 1500
    distance = 0.1
    beta = 0.03  
    gamma = 0.1  
    initial_infected_fraction = 0.5
    num_steps = 10

    population_graph = initialize_population_graph(num_nodes, distance)
    SIR_model_visualization(population_graph, beta, gamma, initial_infected_fraction, num_steps)

SIER MODEL WITH PARTIAL IMMUNITY ALONG WITH AGE-WISE POPULATION DISTRIBUTION

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
from uncertainties import ufloat 

def initialize_population_graph(num_nodes, distance):
    G = nx.random_geometric_graph(num_nodes, distance, seed=896803)
    return G

def initialize_node_states(G, initial_infected_fraction, young_percentage, middle_age_percentage, old_percentage, fully_immune_rates):
    num_infected = int(initial_infected_fraction * G.number_of_nodes())
    node_list = list(G.nodes())

    num_young = int(young_percentage * G.number_of_nodes())
    num_middle_age = int(middle_age_percentage * G.number_of_nodes())
    num_old = int(old_percentage * G.number_of_nodes())
    
    age_groups = ['Young'] * num_young + ['MiddleAge'] * num_middle_age + ['Old'] * num_old
    random.shuffle(age_groups)
    
    infected_nodes = random.sample(node_list, num_infected)
    
    node_states = {
        node: {
            'state': 'I',
            'infection_count': 0,
            'fatality_count': 0,
            'age_group': age_groups[i],
            'fully_immune_rate': fully_immune_rates[age_groups[i]].nominal_value,
        }
        if node in infected_nodes
        else {
            'state': 'S',
            'infection_count': 0,
            'fatality_count': 0,
            'age_group': age_groups[i],
            'fully_immune_rate': fully_immune_rates[age_groups[i]].nominal_value,
        }
        for i, node in enumerate(G.nodes())
    }
    
    return node_states

def plot_sir_graph(G, node_states, title):
    plt.figure(figsize=(30, 30))
    pos = nx.spring_layout(G, seed=42, k=0.1)

    colors = {'S': 'blue', 'I': 'red', 'R': 'green', 'D': 'black', 'Y': 'yellow'}
    node_colors = [colors[node_states[node]['state']] for node in G.nodes()]

    edge_colors = [colors[node_states[u]['state']] for u, v in G.edges()]

    nx.draw_networkx(G, pos, with_labels=False, node_color=node_colors, edge_color=edge_colors, node_size=20, alpha=0.2)

    plt.title(title)
    plt.axis("off")
    plt.show()

def adjust_beta_based_on_neighbors(G, node_states, beta, threshold_low=5, threshold_high=30):
    new_beta_values = {node: beta for node in G.nodes()}

    for node in G.nodes():
        if node_states[node]['state'] == 'I':
            num_neighbors = len(list(G.neighbors(node)))
            if num_neighbors < threshold_low:
                new_beta_values[node] /= 2  
            elif num_neighbors > threshold_high:
                new_beta_values[node] *= 2  

    return new_beta_values

def infect_neighbors(G, node_states, beta_values):
    new_node_states = node_states.copy()
    for node in G.nodes():
        if node_states[node]['state'] == 'I':
            for neighbor in G.neighbors(node):
                if node_states[neighbor]['state'] == 'S' and random.random() < beta_values[node]:
                    new_node_states[neighbor]['state'] = 'I'
                    new_node_states[neighbor]['infection_count'] += 1
    return new_node_states

def recover_nodes(node_states, recovery_rates, fully_immune_rates, fatality_rates, reinfection_rate, max_infections):
    new_node_states = node_states.copy()
    
    for node in node_states:
        if node_states[node]['state'] == 'I':
            age_group = node_states[node]['age_group']
            
            recovery_rate = recovery_rates[age_group].nominal_value
            fatality_rate = fatality_rates[age_group].nominal_value
            
            if random.random() < fatality_rate:
                new_node_states[node]['state'] = 'D'  # Fatality
                new_node_states[node]['fatality_count'] += 1
            elif random.random() < recovery_rate:
                if random.random() < fully_immune_rates[age_group] or node_states[node]['infection_count'] >= max_infections:
                    new_node_states[node]['state'] = 'R'
                else:
                    if random.random() < reinfection_rate.nominal_value:
                        new_node_states[node]['state'] = 'Y'  
                        new_node_states[node]['infection_count'] += 1
                    else:
                        new_node_states[node]['state'] = 'R'  

    return new_node_states

def SIR_model_visualization(G, beta, initial_infected_fraction, num_steps, recovery_rates, fully_immune_rates, fatality_rates, reinfection_rate, max_infections, beta_threshold=30):
    young_percentage = 0.25
    middle_age_percentage = 0.65
    old_percentage = 0.10
    
    node_states = initialize_node_states(G, initial_infected_fraction, young_percentage, middle_age_percentage, old_percentage, fully_immune_rates)
    
    for step in range(num_steps):
        plot_sir_graph(G, node_states, f'Step {step + 1}')
        num_infected = sum(1 for node in G.nodes() if node_states[node]['state'] == 'I')
        num_susceptible = sum(1 for node in G.nodes() if node_states[node]['state'] == 'S')
        num_partially_recovered = sum(1 for node in G.nodes() if node_states[node]['state'] == 'Y')
        num_death = sum(1 for node in G.nodes() if node_states[node]['state'] == 'D')
        num_fully_immune = sum(1 for node in G.nodes() if node_states[node]['state'] == 'R')

        print(f"Step {step + 1}: Infected={num_infected}, Susceptible={num_susceptible}, Partially Recovered={num_partially_recovered}, Death={num_death}, Fully Immune={num_fully_immune}")

        adjusted_beta_values = adjust_beta_based_on_neighbors(G, node_states, beta, beta_threshold)
        node_states = infect_neighbors(G, node_states, adjusted_beta_values)
        node_states = recover_nodes(node_states, recovery_rates, fully_immune_rates, fatality_rates, reinfection_rate, max_infections)

if __name__ == "__main__":
    num_nodes = 2000
    distance = 0.1
    beta = 0.03
    initial_infected_fraction = 0.05
    num_steps = 15
    fully_immune_rates = {'Young': ufloat(0.450, 0.3), 'MiddleAge': ufloat(0.40, 0.3), 'Old': ufloat(0.35, 0.3)}
    fatality_rates = {'Young': ufloat(0.01, 0.0001), 'MiddleAge': ufloat(0.03, 0.002), 'Old': ufloat(0.13, 0.3)}
    reinfection_rate = ufloat(0.057, 0.02)
    max_infections = 5  
    recovery_rates = {'Young': ufloat(0.25, 0.4), 'MiddleAge': ufloat(0.20, 0.4), 'Old': ufloat(0.18, 0.4)}

    population_graph = initialize_population_graph(num_nodes, distance)
    SIR_model_visualization(population_graph, beta, initial_infected_fraction, num_steps, recovery_rates, fully_immune_rates, fatality_rates, reinfection_rate, max_infections)


In [ ]:
#Used for drawing heatmap for a given population using seaborn and random geometric graph from networkx module
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

def population_graph(num_nodes, distance):
    G = nx.random_geometric_graph(num_nodes, distance, seed=896803)
    return G

def draw_node_heatmap(graph):
    pos = nx.get_node_attributes(graph, 'pos')  
    x, y = zip(*pos.values())

    plt.figure(figsize=(20, 20))
    nx.draw(graph, pos=pos, with_labels=False, node_size=20, alpha=0.2)
    sns.kdeplot(x=x, y=y, cmap="Blues", fill=True, thresh=0, levels=1000)
    plt.title("Node Density in the Network Graph")
    plt.show()
if __name__ == "__main__":
    num_nodes = 2000 
    distance = 0.1
    G = population_graph(num_nodes, distance)
    draw_node_heatmap(G)
